Проект прогнозирования погоды на след день: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package


Без Графики. только комманды.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

#from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
#cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

from scipy import stats
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer

import dill,os

In [2]:
data = pd.read_csv("weatherAUS.csv")
data.head(3)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No


In [3]:
# Собираем колонки
columns_numer=['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'WindGustSpeed']
columns_numer_two=['WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm']
#columns_cat_new=['Location_new', 'WindDir9am_new', 'WindDir3pm_new', 'WindGustDir_new']
columns_cat_old=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']
Predict_fich=['RainTomorrow']

In [4]:
# заменяем Итоговую переменнную.
data['RainTomorrow'] = data['RainTomorrow'].map({'Yes': 1, 'No': 0})
data['RainTomorrow'] = data['RainTomorrow'].fillna(0)

In [5]:
# Создаем нескольк классов для Pipeline
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

class ToDF(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = [key]
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return pd.DataFrame(X, columns=self.key)

    
class MyLabelEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key    
    def fit(self, X, y=None):
        return self
    def transform(self, X): 
        labls=LabelEncoder().fit(X[self.key])
        X1=labls.transform(X[self.key])
        return X1

class Blowout(BaseEstimator, TransformerMixin):
    def __init__(self, key=None): 
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X1=X.copy()
        X2=X1[(stats.zscore(X1)<3).all(axis=1)]
        return X2


In [6]:
# обработка столбцов
final_transformers = list()
for cont_col in columns_numer+columns_numer_two:
    num_pipe=Pipeline([
    ('selector', ColumnSelector(key=cont_col)),
    ('to_df', ToDF(key=cont_col)),
    ('imputer', SimpleImputer(strategy='mean'))
    #('to_df', ToDF(key=cont_col))
    #('scaler', StandardScaler())
    ])
    final_transformers.append((cont_col, num_pipe))

    
    
for cont_col in columns_cat_old:
    cat_pipe=Pipeline([
        ('selector', ColumnSelector(key=cont_col)),
        ('to_df1', ToDF(key=cont_col)),
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('to_df2', ToDF(key=cont_col)),
        ('one', MyLabelEncoder(key=cont_col)),
        ('to_df3', ToDF(key=cont_col))
    ])
    final_transformers.append((cont_col, cat_pipe))
    
feats = FeatureUnion(final_transformers)

    

In [10]:
# ИТОГОВЫЙ Pipeline
my_pipe=Pipeline([
    #('transform', transformer),
    ('feats', feats),
    #('Blowout', Blowout()),
    #('scaler', StandardScaler()),
    ('xgbc', XGBClassifier(objective='binary:logistic'))
])

In [11]:
x, y=data[columns_numer+columns_numer_two+columns_cat_old], data[Predict_fich]

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
my_pipe.fit(x_train, y_train)

E:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Pipeline(steps=[('feats',
                 FeatureUnion(transformer_list=[('MinTemp',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelector(key='MinTemp')),
                                                                 ('to_df',
                                                                  ToDF(key=['MinTemp'])),
                                                                 ('imputer',
                                                                  SimpleImputer())])),
                                                ('MaxTemp',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelector(key='MaxTemp')),
                                                                 ('to_df',
                                                                  ToDF(key=['MaxTemp'])),
        

In [14]:
predictions=my_pipe.predict_proba(x_test)
#predictions[:, 1]

In [15]:
roc_auc_score(y_score=predictions[:, 1][:], y_true=y_test.iloc[:])

0.8857866936751551

In [17]:
with open("pipeline.dill", "wb") as f:
    dill.dump(my_pipe, f)